# The following are the parameter spaces used for evaluations

## The parameter search space used by Tuner for xgboost, KNN and SVM in the Section IV-A2.

In [1]:
from scipy.stats import uniform
from mango.domain.distribution import loguniform

param_svm = {'gamma':uniform(0.1, 4),
             'C':loguniform(-7, 10)}

param_xgboost = {'learning_rate':uniform(0, 1),
                 'gamma':uniform(0, 5),
                 'max_depth':range(1,11),
                 'n_estimators':range(1,301),
                 'booster':['gbtree','gblinear','dart']}
               
param_knn = {'n_neighbors':range(1, 51),
             'algorithm':['auto','ball_tree','kd_tree',
             'brute']}

## The parameter search space used by MetaTuner for neural network, decision tree, xgboost, KNN and SVM in the Section IV-A3. The neural network has one hidden layer with number of hidden nodes specified in the search space. 

In [2]:
from scipy.stats import uniform
from mango.domain.distribution import loguniform

param_nn = {'type':'clf_nn',
            'num_of_nodes':range(10, 101)}

param_dtree = {'type':'clf_dtree',
               'max_features':['auto', 'sqrt', 'log2'],
               'max_depth':range(1,21),     
               'splitter':['best','random'],
               'criterion':['gini','entropy']}
             
param_svm = {'type':'clf_svm',
             'gamma':uniform(0.1, 4),
             'C':loguniform(-7, 10)}

param_xgboost = {'type':'clf_xgboost',
                 'learning_rate':uniform(0, 1),
                 'gamma':uniform(0, 5),
                 'max_depth':range(1,21),
                 'n_estimators':range(1,11),
                 'booster':['gbtree','gblinear','dart']}
               
param_knn = {'type':'clf_knn',
             'n_neighbors': range(1, 51),
             'algorithm':['auto','ball_tree','kd_tree', 
             'brute']}


# The parameter space used for the TCN network in the Section IV:D Case Study: Network Architecture Search for TinyML Platforms

In [6]:
import itertools
import numpy as np

min_layer = 3
max_layer = 8
a_list = [1,2,4,8,16,32,64,128,256]
all_combinations = []
dil_list = []
for r in range(len(a_list) + 1):
    combinations_object = itertools.combinations(a_list, r)
    combinations_list = list(combinations_object)
    all_combinations += combinations_list
all_combinations = all_combinations[1:]
for item in all_combinations:
    if(len(item) >= min_layer and len(item) <= max_layer):
        dil_list.append(list(item))

In [7]:
param_dict = {
    'nb_filters': range(2,64),
    'kernel_size': range(2,16),
    'dropout_rate': np.arange(0.0,0.5,0.1),
    'use_skip_connections': [True, False],
    'norm_flag': np.arange(0,1),
    'dil_list': dil_list
}

In [8]:
# A sample TCN is constructed using the above space using Keras TCN library as:

def objective_NN(epochs=500,nb_filters=32,kernel_size=7,dilations=[1, 2, 4, 8, 16, 32, 64, 128],dropout_rate=0,
                 use_skip_connections=False,norm_flag=0):
    
    batch_size, timesteps, input_dim = 256, window_size, 6
    i = Input(shape=(timesteps, input_dim))
    
    if(norm_flag==1):
        m = TCN(nb_filters=nb_filters,kernel_size=kernel_size,dilations=dilations,dropout_rate=dropout_rate,
                use_skip_connections=use_skip_connections,use_batch_norm=True)(i)
    else:
        m = TCN(nb_filters=nb_filters,kernel_size=kernel_size,dilations=dilations,dropout_rate=dropout_rate,
                use_skip_connections=use_skip_connections)(i)        
        
    m = tf.reshape(m, [-1, nb_filters, 1])

    m = MaxPooling1D(pool_size=(2))(m)
    m = Flatten()(m)
    m = Dense(32, activation='linear', name='pre')(m)
    output1 = Dense(1, activation='linear', name='velx')(m)
    output2 = Dense(1, activation='linear', name='vely')(m)
    model = Model(inputs=[i], outputs=[output1, output2])
    opt = tf.keras.optimizers.Adam()
    model.compile(loss={'velx': 'mse','vely':'mse'},optimizer=opt)  
    
    return model